In [ ]:
#using Pkg
#Pkg.add("GeneNetworkAPI")

In [1]:
using Helium;
using DataFrames;
using CSV;
using GeneNetworkAPI;
using DelimitedFiles;

In [2]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis/HSNIH-Palmer"

In [3]:
include("../../src/readData.jl");

In [4]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis/HSNIH-Palmer"

In [5]:
pheno_10441 = get_pheno("HSNIH-Palmer", "10441");

In [6]:
size(pheno_10441)

(4099, 4)

In [7]:
pheno_10441[1:6, :]

,data_id,sample_name,sample_name_2,value
,Int64,String,String,Float64
1,41013852,00071F4FAF,00071F4FAF,67.0
2,41013852,00071FB160,00071FB160,32.0
3,41013852,0007207A73,0007207A73,70.0
4,41013852,0007213055,0007213055,45.0
5,41013852,0007213615,0007213615,59.0
6,41013852,00072136BA,00072136BA,38.0


In [8]:
geno_filename = "HSNIH-Palmer.he";

In [9]:
geno_mat = readhe(geno_filename);

In [10]:
size(geno_mat)

(134918, 6147)

## Check for infinite values in the genotype array:

In [11]:
sum(geno_mat)

NaN

In [12]:
NaN_to_remove = zeros(size(geno_mat, 1));

In [13]:
for p = 1:size(geno_mat, 1)
    
    if any(!isfinite, geno_mat[p, :])
        
        NaN_to_remove[p] = 1
        
    end
    
end

In [14]:
sum(NaN_to_remove) # 282 markers have infinite values; we simply remove those rows

282.0

In [15]:
geno_mat = geno_mat[NaN_to_remove .!= 1, :];
size(geno_mat)

(134636, 6147)

In [16]:
geno_supp = Helium.getsupp(geno_filename);
size(geno_supp)

(134919, 5)

In [17]:
geno_supp[1:10, :]

10×5 Matrix{String}:
 "SNPID"  "rsid"          "position"  "alleleA"  "alleleB"
 "1"      "chr1:55365"    "55365"     "A"        "T"
 "1"      "chr1:666374"   "666374"    "T"        "C"
 "1"      "chr1:666382"   "666382"    "A"        "T"
 "1"      "chr1:666394"   "666394"    "G"        "A"
 "1"      "chr1:669529"   "669529"    "C"        "T"
 "1"      "chr1:669562"   "669562"    "T"        "A"
 "1"      "chr1:671466"   "671466"    "T"        "C"
 "1"      "chr1:759319"   "759319"    "T"        "C"
 "1"      "chr1:1134030"  "1134030"   "A"        "G"

In [18]:
geno_supp_header = geno_supp[1, :];
geno_supp = geno_supp[2:end, :];

In [19]:
geno_supp = geno_supp[NaN_to_remove .!= 1, :]

134636×5 Matrix{String}:
 "1"   "chr1:55365"      "55365"     "A"  "T"
 "1"   "chr1:666374"     "666374"    "T"  "C"
 "1"   "chr1:666382"     "666382"    "A"  "T"
 "1"   "chr1:666394"     "666394"    "G"  "A"
 "1"   "chr1:669529"     "669529"    "C"  "T"
 "1"   "chr1:669562"     "669562"    "T"  "A"
 "1"   "chr1:671466"     "671466"    "T"  "C"
 "1"   "chr1:759319"     "759319"    "T"  "C"
 "1"   "chr1:1134030"    "1134030"   "A"  "G"
 "1"   "chr1:1139210"    "1139210"   "T"  "C"
 "1"   "chr1:1143318"    "1143318"   "T"  "C"
 "1"   "chr1:1151014"    "1151014"   "G"  "C"
 "1"   "chr1:1151294"    "1151294"   "T"  "C"
 ⋮                                        
 "20"  "chr20:55641278"  "55641278"  "C"  "A"
 "20"  "chr20:55644746"  "55644746"  "C"  "G"
 "20"  "chr20:55659520"  "55659520"  "G"  "A"
 "20"  "chr20:55716379"  "55716379"  "T"  "G"
 "20"  "chr20:55744179"  "55744179"  "A"  "C"
 "20"  "chr20:55773139"  "55773139"  "T"  "G"
 "20"  "chr20:55818290"  "55818290"  "G"  "A"
 "20"  "chr2

In [20]:
geno_sampleID = Helium.getcolnames(geno_filename);

In [21]:
pheno_sampleID = pheno_10441.sample_name;

In [22]:
match_id = Array{Int, 1}(undef, 6147);

In [23]:
@time begin 
    for i in 1:6147
    
    indicator = 0;
    for j in 1:4099
        if geno_sampleID[i] == pheno_sampleID[j]
            indicator = indicator + 1;
        end
    end
    
    match_id[i] = indicator;
end
end

  1.942802 seconds (45.16 M allocations: 689.133 MiB)


In [24]:
sum(match_id)

4094

In [25]:
typeof(match_id)

Vector{Int64} (alias for Array{Int64, 1})

In [26]:
samples_in_common = geno_sampleID[match_id .==1];

In [27]:
match_id_pheno = Array{Int, 1}(undef, 4099);

In [28]:
@time begin 
    for i in 1:length(match_id_pheno)
    
    indicator = 0;
    for j in 1:length(samples_in_common)
        if pheno_sampleID[i] == samples_in_common[j]
            indicator = indicator + 1;
        end
    end
    
    match_id_pheno[i] = indicator;
end
end

  2.468816 seconds (46.17 M allocations: 960.750 MiB)


In [29]:
sum(match_id_pheno)

4094

In [30]:
size(geno_mat)

(134636, 6147)

In [31]:
size(pheno_10441)

(4099, 4)

In [32]:
## subset geno:
geno_common = geno_mat[:, match_id .== 1];
## subset pheno:
pheno_common = pheno_10441[match_id_pheno .== 1, :];

In [33]:
size(pheno_common)

(4094, 4)

In [34]:
size(geno_common)

(134636, 4094)

Finally, we got a sample genofile with 4094 individuals, and 134918 genotype markers.

In [35]:
n = size(geno_common, 2)

4094

In [36]:
mean_freq = sum(geno_common; dims = 2)./(2*n);

In [37]:
less_common_markers = map(x -> (x > 0.90) || (x < 0.10), mean_freq)[:, 1];

In [38]:
typeof(less_common_markers)

Vector{Bool} (alias for Array{Bool, 1})

In [39]:
size(geno_common, 1) - sum(less_common_markers)

102279

In [40]:
geno_processed = permutedims(geno_common[less_common_markers .!= 1, :]);

In [41]:
pheno_processed = reshape(pheno_common[:, 4], :, 1);

In [42]:
size(geno_processed)

(4094, 102279)

In [43]:
size(pheno_processed)

(4094, 1)

## Run Gemma:

In [ ]:
gemma = "/home/xyu/software/GEMMA/gemma-0.98.5-linux-static-AMD64";

In [ ]:
run(`$gemma -h`)

In [ ]:
pwd()

In [ ]:
include("../../src/readData.jl");

In [ ]:
#=
outfile = "pheno.txt"
open(outfile, "w") do f
    for i in pheno_processed
        println(f, i)
    end
end
=#

In [ ]:
#=
outfile = "geno.txt"
open(outfile, "w") do f
    for i in pheno_processed
        println(f, i)
    end
end
=#

In [44]:
geno_supp

134636×5 Matrix{String}:
 "1"   "chr1:55365"      "55365"     "A"  "T"
 "1"   "chr1:666374"     "666374"    "T"  "C"
 "1"   "chr1:666382"     "666382"    "A"  "T"
 "1"   "chr1:666394"     "666394"    "G"  "A"
 "1"   "chr1:669529"     "669529"    "C"  "T"
 "1"   "chr1:669562"     "669562"    "T"  "A"
 "1"   "chr1:671466"     "671466"    "T"  "C"
 "1"   "chr1:759319"     "759319"    "T"  "C"
 "1"   "chr1:1134030"    "1134030"   "A"  "G"
 "1"   "chr1:1139210"    "1139210"   "T"  "C"
 "1"   "chr1:1143318"    "1143318"   "T"  "C"
 "1"   "chr1:1151014"    "1151014"   "G"  "C"
 "1"   "chr1:1151294"    "1151294"   "T"  "C"
 ⋮                                        
 "20"  "chr20:55641278"  "55641278"  "C"  "A"
 "20"  "chr20:55644746"  "55644746"  "C"  "G"
 "20"  "chr20:55659520"  "55659520"  "G"  "A"
 "20"  "chr20:55716379"  "55716379"  "T"  "G"
 "20"  "chr20:55744179"  "55744179"  "A"  "C"
 "20"  "chr20:55773139"  "55773139"  "T"  "G"
 "20"  "chr20:55818290"  "55818290"  "G"  "A"
 "20"  "chr2

In [45]:
geno_supp_processed = geno_supp[less_common_markers .!= 1, :];

In [46]:
size(geno_supp_processed)

(102279, 5)

In [47]:
@time geno_processed = permutedims(geno_processed);

  7.452627 seconds (2 allocations: 3.120 GiB)


In [52]:
size(geno_processed)

(102279, 4094)

In [53]:
marker_names = geno_supp_processed[:, 2];
minor_allele = geno_supp_processed[:, 4];
major_allele = geno_supp_processed[:, 5];
@time geno_prob_bimbam = hcat(marker_names, minor_allele, major_allele);

  0.002744 seconds (3 allocations: 2.341 MiB)


In [54]:
geno_prob_bimbam = hcat(geno_prob_bimbam, geno_processed);

In [63]:
geno_prob_bimbam[1:10, :]

10×4097 Matrix{Any}:
 "chr1:759319"   "T"  "C"  0.99   0.997  …  0.007  0.007    0.99603  0.007
 "chr1:1154955"  "G"  "A"  2.0    1.0       1.0    2.0      2.0      2.0
 "chr1:1649028"  "G"  "A"  0.002  0.003     0.003  0.002    1.0      0.002
 "chr1:1649332"  "C"  "T"  0.0    0.0       0.0    0.0      1.0      0.0
 "chr1:1651967"  "A"  "T"  0.002  0.002     0.002  0.002    1.0      0.002
 "chr1:1671721"  "G"  "A"  0.003  0.003  …  0.003  0.003    0.967    0.003
 "chr1:1685077"  "C"  "T"  1.997  0.998     0.999  1.997    1.939    1.997
 "chr1:1691966"  "C"  "T"  1.998  0.999     1.0    1.998    0.999    1.998
 "chr1:1754257"  "C"  "T"  2.0    1.0       1.0    2.0      1.0      2.0
 "chr1:1755622"  "G"  "C"  0.038  0.022     0.092  0.07605  1.097    0.087

In [64]:
size(geno_prob_bimbam)

(102279, 4097)

In [65]:
pheno_filename = "pheno.txt";

In [66]:
geno_filename = "geno.txt";

In [67]:
#writedlm(pheno_filename, pheno_processed);

In [69]:
#writeToFile(geno_prob_bimbam, geno_filename);

In [ ]:
# @time run(`$gemma -g $geno_filename -p $pheno_filename -gk 1 -o kinship`)

In [ ]:
kinship_Gc = CSV.read("output/kinship.cXX.txt", DataFrame, delim = '\t', header = false);

In [ ]:
size(kinship_Gc)

In [ ]:
pheno_processed

In [ ]:
kinshipMat_Gc = Matrix(kinship_Gc); 

In [ ]:
size(pheno_processed)

In [ ]:
geno_processed_transpose = permutedims(geno_processed);

In [ ]:
size(geno_processed_transpose)

In [ ]:
using DelimitedFiles
using LinearAlgebra
using Optim
using Distributions
using Test
using BenchmarkTools
using Random
using Plots
using Profile
using Distributed

In [ ]:
# writedlm("pheno.csv", pheno_processed);
# writedlm("geno.csv", geno_processed_transpose);

In [ ]:
# @time results_bulklmm_alt = scan(pheno_processed, geno_processed_transpose, kinshipMat_Gc; reml = false, method = "null");

In [ ]:
# @time run(`$gemma -g $geno_filename -p $pheno_filename -k output/kinship.cXX.txt -lmm 2 -o gemma_results.txt`)

## Run BulkLMM:

In [ ]:
# @time kinship_BulkLMM = calcKinship(geno_processed)

In [ ]:
# size(kinship_BulkLMM)

In [ ]:
# pheno_common

In [ ]:
# @time results_BulkLMM = scan(pheno_processed, geno_processed, kinship_BulkLMM; reml = false, method = "alt")